In [2]:
import boto3
import json
import base64
import time
import re
from botocore.exceptions import ClientError
import logging

bedrock_runtime = boto3.client(
    service_name="bedrock-runtime",
    region_name="us-east-1",
)

bedrock = boto3.client(
    service_name='bedrock', 
    region_name='us-east-1'
)

brt = boto3.client(service_name='bedrock-runtime')

def decoder(data):
    decodedBytes = base64.b64decode(data)
    decodedStr = decodedBytes.decode("ascii") 
    json_str=json.loads(decodedStr)
    return json_str

##Processing data to fetch role and content
def data_preprocessing(data):
    #data = json_data['Records'][0]["body"]
    convo = ""
    content = json.loads(data["body"])["body"]["transcript"][0]["Content"]
    role = json.loads(data["body"])["body"]["transcript"][0]["ParticipantRole"]
    convo = convo + role + " : " + content
    #parsed_data = json.loads(data)
    #content = data#["body"]#["transcript"]#[0]#["Content"]
    #role = data["body"]["transcript"][0]["ParticipantRole"]
    
    #convo = ""
    #for i in range(len(transcription['transcriptions'])):
        #convo = convo + transcription['transcriptions'][i]['ParticipantRole'] + ": " + transcription['transcriptions'][i]['Content']
        #convo += "\n"
    #role = transcription["Segments"][0]["Transcript"]["ParticipantRole"]
    #content = transcription["Segments"][0]["Transcript"]["Content"]
    #convo = convo + role + " : " + content
    
    return convo

def data_postprocessing(data):
    result = ""
    start_index = data.find("{")
    end_char_indices = [i.start() for i in re.finditer("}",data)]
    end_index = end_char_indices[len(end_char_indices)-1]
    result = data[start_index:end_index+1]
    
    return result

    
#Defining function to connect to Bedrock LLM
def load_claude2(bedrock_runtime , prompt , temp , top_p,top_k):
    try:
        body = {
            "prompt": prompt,
            "temperature": temp,
            "top_p": top_p,
            "top_k":top_k,
            "max_tokens_to_sample": 1000
            }

        response = bedrock_runtime.invoke_model(
            modelId="anthropic.claude-v2", body=json.dumps(body), accept="application/json", contentType="application/json"
                 )
        
        response_body = json.loads(response["body"].read())
        completion = response_body.get("completion")

        return completion

    except ClientError:
        logging.error("Couldn't invoke Llama 2")
        raise

#Defining LLM function for the prompt generator for entity extraction  


insurance_statistics = '''
{
  "ANTHEM INC.": {
    "ALTSCRIPTS": "5 days",
    "RECEPTRX": "14 days",
    "GENTRY HEALTH": "15 days",
    "ALLIANCE RX": "22 days",
    "BIOPLUS": "26 days"
  },
  "BLUE CROSS/BLUE SHIELD": {
    "US BIO": "6 days",
    "RECEPTRX": "24 days",
    "ALLIANCE RX": "43 days",
    "KROGER SPECIALTY PHARMACY": "45 days",
    "ACARIA": "45 days"
  },
  "CENTENE CORPORATION": {
    "GENTRY HEALTH": "4 days",
    "CENTERWELL": "5 days",
    "FAIRVIEW SP": "5 days",
    "LUMICERA": "16 days",
    "BIOPLUS": "22 days"
  },
  "CIGNA": {
    "AMBER": "13 days",
    "CENTURY SPECIALTY": "14 days",
    "ALTSCRIPTS": "17 days",
    "KROGER SPECIALTY PHARMACY": "26 days",
    "LUMICERA": "30 days"
  },
  "CVS CAREMARK RX": {
    "FAIRVIEW SP": "6 days",
    "CENTURY SPECIALTY": "11 days",
    "ALTSCRIPTS": "14 days",
    "AMBER": "17 days",
    "ELIXIR": "28 days"
  },
  "CVS HEALTH (AETNA)": {
    "CENTURY SPECIALTY": "11 days",
    "ALTSCRIPTS": "12 days",
    "ELIXIR": "16 days",
    "GENTRY HEALTH": "17 days",
    "ALLIANCE RX": "28 days"
  },
  "DST PHARMACY SOLUTIONS": {
    "ARDON": "3 days"
  },
  "HUMANA INC.": {
    "LUMICERA": "8 days",
    "ALTSCRIPTS": "15 days",
    "ALLIANCE RX": "19 days",
    "FAIRVIEW SP": "20 days",
    "BIOPLUS": "21 days"
  },
  "OPTUMRX": {
    "ELIXIR": "17 days",
    "NOBLE HEALTH SERVICES": "24 days",
    "BIOPLUS": "28 days",
    "ACARIA": "30 days",
    "CENTERWELL": "32 days"
  },
  "RELAYHEALTH": {
    "RECEPTRX": "4 days"
  },
  "UNITEDHEALTHCARE": {
    "CENTURY SPECIALTY": "11 days",
    "GENTRY HEALTH": "19 days",
    "LUMICERA": "19 days",
    "FAIRVIEW SP": "26 days",
    "ALLIANCE RX": "28 days"
  }
}
'''

entities = "name of patient, status of insurance, insurance number, demographic details etc."
def enrollment_prompt_generator(conversation,entities):
    prompt_claude = """Human: \" """ + conversation + """ \"
 
The above conversation is an automated transcript between a call centre agent and an insurance subscriber or 
patiet. I want to extract few key entities like \" """ + entities + """ \"". All or some information may be present in this transcript.
Extract the entities that you are able to find from this piece of call transcript.
 
The output would be a structured json with only the extracted fields. Just print the exact output without any extra sentences at the end or beggining. 
No need to print any extra text. Also do not generate an answer if that is not found in the transcript.
 
Assistant:
"""
    return prompt_claude

def insights_prompt_generator(insurance_provider,insurance_statistics):
    prompt_claude = """Human: 
 
You are Agent assist tracking the Patient and agent conversation and help the agent recommend meaningful insights on the 
insurance and insurance details related insights like for example suggesting which pharmacy to select based on the
patient's Insurance provider using metrics like how soon the pharmacy dispenses the medication to the patient.The lesser the
number of days to dispense the medication,the higher are the chances of recommendation of that pharmacy.

The patients's insurance provider is  \" """ + insurance_provider + """ \" and use the following json data to provide the insights:
\" """ + insurance_statistics + """ \".

In the Json Data,the keys represent the insurance provider and the value represents the pharmacy company and the number of days
it takes to dispense the medication to the patient
 
Assistant:
"""
    return prompt_claude

# Lambda handler to intgerate with AWS
def lambda_handler(event,context):
    final_transcript = ""
    for i in range(len(event['Records'])):
        final_transcript += "\n" + data_preprocessing(event['Records'][i])
    prompt_enrollment = enrollment_prompt_generator(final_transcript,entities)
    enrollment_data = load_claude2(bedrock_runtime,prompt_enrollment,0,0.9,1)
    json_data = data_postprocessing(enrollment_data)
    enrollment_json_object = json.loads(json_data)
    keys = enrollment_json_object.keys()
    for val in keys:
        if val == "insurance_provider":
            insurance_provider = enrollment_json_object[val]
        else:
            insurance_provider = " "
    insights_prompt = insights_prompt_generator(insurance_provider,insurance_statistics)
    insights = load_claude2(bedrock_runtime,insights_prompt,0.5,0.9,1)
    
    return {"statusCode": 200,"body": json.dumps(insights)}

# Testing

In [1]:
data = {
  "Records": [
    {
      "messageId": "355c3548-1f03-4c5e-b241-3c6e4bed5601",
      "receiptHandle": "AQEBtGG/BXpQS9fewvrud7Ct+0Pm1fWqFdXUutYN2NKU5Ohukwi/rZLaaKD7mNkMv619WOI1ao9LwvhDO573DROL+wUCQQz1ejI+BaTYWKs629P7sBa8/ND4Bv3SfKPe4p+rMALYuANfO/XoeN6lfjvkNGcSWiyO5PrVyyEPPIKSIg/d/p2viccwRB3s7Q/0TwIxK3iYkSkHoUAOYdcCEQQU1Ia6srdP0D4RCtFXPh4S21F4IljNVF5DYiEFYGX78bPdTVluBqZm9xpjEYw+fOl8YrsIM0p5S9vCBQhouj5rC8Xo8dbrrUo6SAwJqmxWSckmHMda14eyA6zyomtz2xzmxCwdzUVvRMa4J9eMPB+u941W10aK7CP/PvqziaO28aDNuStE6KLOF/2VmgfK9eXF5w==",
      "body": "{\"stream\":\"TRANSCRIPT\",\"streamConnectionId\":\"149edb05-7fd0-48b0-a6e9-3e7b49a0b8d7\",\"body\":{\"transactionId\":\"50b1f7fb-b502-42fa-95f2-281ed6006efe\",\"transcript\":[{\"ParticipantId\":\"CUSTOMER\",\"ParticipantRole\":\"CUSTOMER\",\"Content\":\"Sure,my insurance is with Anthem INC.\",\"BeginOffsetMillis\":399340,\"EndOffsetMillis\":399745,\"Id\":\"f10ada6b-dfeb-4569-8b2c-f4e83d60efbc\",\"Sentiment\":\"NEUTRAL\",\"IssuesDetected\":[]}]}}",
      "attributes": {
        "ApproximateReceiveCount": "1",
        "AWSTraceHeader": "Root=1-65d48ed1-196e903e02d83fbd5c5a23bb;Parent=07e7ba1a0a2f4dae;Sampled=0;Lineage=8448a41f:0",
        "SentTimestamp": "1708429009330",
        "SenderId": "AROAVSPTL2UI67PVNI4RP:CH_Kinesis_Transcriber",
        "ApproximateFirstReceiveTimestamp": "1708429009331"
      },
      "messageAttributes": {},
      "md5OfBody": "3488de905e211729d102b02f70a21683",
      "eventSource": "aws:sqs",
      "eventSourceARN": "arn:aws:sqs:us-east-1:383299343633:ch-agent-assist-sqs",
      "awsRegion": "us-east-1"
    },
    {
      "messageId": "355c3548-1f03-4c5e-b241-3c6e4bed5601",
      "receiptHandle": "AQEBtGG/BXpQS9fewvrud7Ct+0Pm1fWqFdXUutYN2NKU5Ohukwi/rZLaaKD7mNkMv619WOI1ao9LwvhDO573DROL+wUCQQz1ejI+BaTYWKs629P7sBa8/ND4Bv3SfKPe4p+rMALYuANfO/XoeN6lfjvkNGcSWiyO5PrVyyEPPIKSIg/d/p2viccwRB3s7Q/0TwIxK3iYkSkHoUAOYdcCEQQU1Ia6srdP0D4RCtFXPh4S21F4IljNVF5DYiEFYGX78bPdTVluBqZm9xpjEYw+fOl8YrsIM0p5S9vCBQhouj5rC8Xo8dbrrUo6SAwJqmxWSckmHMda14eyA6zyomtz2xzmxCwdzUVvRMa4J9eMPB+u941W10aK7CP/PvqziaO28aDNuStE6KLOF/2VmgfK9eXF5w==",
      "body": "{\"stream\":\"TRANSCRIPT\",\"streamConnectionId\":\"149edb05-7fd0-48b0-a6e9-3e7b49a0b8d7\",\"body\":{\"transactionId\":\"50b1f7fb-b502-42fa-95f2-281ed6006efe\",\"transcript\":[{\"ParticipantId\":\"AGENT\",\"ParticipantRole\":\"AGENT\",\"Content\":\"Thank you,can you share your policy number? \",\"BeginOffsetMillis\":399340,\"EndOffsetMillis\":399745,\"Id\":\"f10ada6b-dfeb-4569-8b2c-f4e83d60efbc\",\"Sentiment\":\"NEUTRAL\",\"IssuesDetected\":[]}]}}",
      "attributes": {
        "ApproximateReceiveCount": "1",
        "AWSTraceHeader": "Root=1-65d48ed1-196e903e02d83fbd5c5a23bb;Parent=07e7ba1a0a2f4dae;Sampled=0;Lineage=8448a41f:0",
        "SentTimestamp": "1708429009330",
        "SenderId": "AROAVSPTL2UI67PVNI4RP:CH_Kinesis_Transcriber",
        "ApproximateFirstReceiveTimestamp": "1708429009331"
      },
      "messageAttributes": {},
      "md5OfBody": "3488de905e211729d102b02f70a21683",
      "eventSource": "aws:sqs",
      "eventSourceARN": "arn:aws:sqs:us-east-1:383299343633:ch-agent-assist-sqs",
      "awsRegion": "us-east-1"
    }
  ]
}

In [5]:
context = " "
print(lambda_handler(data,context)["body"])

" Based on the patient's insurance provider being Anthem Inc., here are the recommended pharmacies and time to dispense medication:\n\nI would recommend AltScripts pharmacy since they have the fastest time to dispense medication for Anthem Inc. at 5 days. \n\nThe other top recommended pharmacies are:\n\n- ReceptRx: 14 days\n- Gentry Health: 15 days  \n\nAltScripts has the fastest dispensing time compared to other pharmacies that work with Anthem Inc., so they would be my top recommendation for this patient. The quicker the medication can be dispensed, the better it is for the patient's health and treatment."
